# Código

### Preambulo

In [ ]:
#para graficos interativos
#%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.colors as colors
plt.style.use('seaborn-whitegrid')
#para imagens estaticas embebidas
%matplotlib inline


import numpy as np
import pandas as pd
import datetime as dtm
import re
import scipy as spy
import glob
import csv

from matplotlib import ticker, cm
from scipy import integrate
from scipy.special import sph_harm
from scipy import interpolate
from numpy import array, linspace, arange, zeros, ceil, amax, amin, argmax, argmin, abs, average
from numpy import polyfit, polyval, seterr, trunc, mean
from numpy.linalg import norm

### Funções

In [ ]:
##### Background LINEAR ####################

DEBUG = False

def linearBG(x, y):
    """ P = specs.preedge_calculate(x,y)
    Calculates the best-fit linear pre-edge for a dataset (x,y). Finds the biggest peak,
    then finds the pre-edge region using a sequence of linear fits starting from the end
    point.
    """

    # Make sure we've been passed arrays and not lists.
    x = array(x)
    y = array(y)

    # Sanity check: Do we actually have data to process here?
    if not (x.any() and y.any()):
        print('specs.preedge_calculate: One of the arrays x or y is empty. Returning zero background.')
        return zeros(x.shape)

    # Next ensure the energy values are *decreasing* in the array,
    # if not, reverse them.
    if x[0] < x[-1]:
        is_reversed = True
        x = x[::-1]
        y = y[::-1]
    else:
        is_reversed = False

    # Locate the biggest peak.
    maxidx = abs(y - amax(y)).argmin()

    # Find the gradient of every possible linear fit between the lowest binding energy
    # and the biggest peak.
    grads = []
    for i in range(2, len(x) - maxidx):
        # Best linear fit to the last i values
        xs = x[-i:]
        ys = y[-i:]
        #p = polyfit(xs,ys,1)
        #grads.append(p[0])
        # Try a new algorithm that should be faster than polyfit
        xs = xs - mean(xs)
        ys = ys - mean(ys)
        grads.append((xs * ys).sum() / (xs * xs).sum())

    # Differentiate the gradient array.
    dgrads = []
    for i in range(len(grads) - 1):
        dgrads.append(grads[i + 1] - grads[i])
    dgrads = array(dgrads)

    # We might not have actually accumulated anything if the maximum is near the
    # edge (like in a survey scan - the SE background is very big). So, may have
    # to return a zero background.
    if not dgrads.any():
        print('specs.preedge_calculate: No pre-edge gradients. The spectrum must be very large at the low kinetic energy end. Returning zero background.')
        return zeros(x.shape)

    # Find the minimum index of the absolute of the gradient of gradients.
    mingrad = abs(dgrads).argmin()

    # Make a best linear fit from this number of pre-edge points, generate linear
    # pre-edge.
    p = polyfit(x[-mingrad:], y[-mingrad:], 1)

    if is_reversed:
        return polyval(p, x)[::-1]
    else:
        return polyval(p, x)
    

In [ ]:
##### Background SHIRLEY ####################

def shirley(x, y, tol=1e-5, maxit=10):
    """ S = specs.shirley_calculate(x,y, tol=1e-5, maxit=10)
    Calculate the best auto-Shirley background S for a dataset (x,y). Finds the biggest peak
    and then uses the minimum value either side of this peak as the terminal points of the
    Shirley background.
    The tolerance sets the convergence criterion, maxit sets the maximum number
    of iterations.
    """

    # Make sure we've been passed arrays and not lists.
    x = array(x)
    y = array(y)

    # Sanity check: Do we actually have data to process here?
    if not (x.any() and y.any()):
        print ("specs.shirley_calculate: One of the arrays x or y is empty. Returning zero background.")
        return zeros(x.shape)

    # Next ensure the energy values are *decreasing* in the array,
    # if not, reverse them.
    if x[0] < x[-1]:
        is_reversed = True
        x = x[::-1]
        y = y[::-1]
    else:
        is_reversed = False

    # Locate the biggest peak.
    maxidx = abs(y - amax(y)).argmin()
    #print('valor:',maxidx)

    # It's possible that maxidx will be 0 or -1. If that is the case,
    # we can't use this algorithm, we return a zero background.
    if maxidx == 0 or maxidx >= len(y) - 1:
        print("specs.shirley_calculate: Boundaries too high for algorithm: returning a zero background.")
        return zeros(x.shape)

    # Locate the minima either side of maxidx.
    lmidx = abs(y[0:maxidx] - amin(y[0:maxidx])).argmin()
    rmidx = abs(y[maxidx:] - amin(y[maxidx:])).argmin() + maxidx
    xl = x[lmidx]
    yl = y[lmidx]
    xr = x[rmidx]
    yr = y[rmidx]

    # Max integration index
    imax = rmidx - 1

    # Initial value of the background shape B. The total background S = yr + B,
    # and B is equal to (yl - yr) below lmidx and initially zero above.
    B = zeros(x.shape)
    B[:lmidx] = yl - yr
    Bnew = B.copy()

    it = 0
    while it < maxit:
        if DEBUG:
            print("Shirley iteration: ", it)
        # Calculate new k = (yl - yr) / (int_(xl)^(xr) J(x') - yr - B(x') dx')
        ksum = 0.0
        for i in range(lmidx, imax):
            ksum += (x[i] - x[i + 1]) * 0.5 * (y[i] + y[i + 1]
                                               - 2 * yr - B[i] - B[i + 1])
        k = (yl - yr) / ksum
        # Calculate new B
        for i in range(lmidx, rmidx):
            ysum = 0.0
            for j in range(i, imax):
                ysum += (x[j] - x[j + 1]) * 0.5 * (y[j] +
                                                   y[j + 1] - 2 * yr - B[j] - B[j + 1])
            Bnew[i] = k * ysum
        # If Bnew is close to B, exit.
        if norm(Bnew - B) < tol:
            B = Bnew.copy()
            break
        else:
            B = Bnew.copy()
        it += 1

    if it >= maxit:
        print("specs.shirley_calculate: Max iterations exceeded before convergence.")
    if is_reversed:
        return (yr + B)[::-1]
    else:
        return yr + B 

In [ ]:
##### plot functions #################### 

def Stgraf(x, y, st='o', xli=(400,610), xlab='BE (eV)', ylab='Intensidade ($10^2$ cps)'):
    """ Funcao que plota o primeiro grafico, sem nenhum tratamento.
    """ 
    plt.plot(x, y/100,st, markersize=6,linewidth=3, markeredgecolor='black', alpha=0.5)
    plt.xlim(xli)
    
    plt.xlabel(xlab, fontsize=24)
    plt.ylabel (ylab, fontsize=24)
    plt.grid(True,color ='lightgray',linestyle='--',linewidth='1')
    plt.tick_params('both', labelsize=20)
    plt.savefig('stgraf_320eV.jpeg')
    
    return plt.show()
#####

def Segraf(x,y1,y2, xli=(400,610)):
    """ Funcao que plota o segundo grafico, onde e mostrada a curva de background linear.
    """
    
    plt.plot(x, y1)
    plt.plot(x,y2)
    plt.xlim(xli)
    
    return plt.show()
#####    

def Tgraf(x, y1, y2, sta='-k', stb='-r', xli=(400,610),xlab='BE (eV)', ylab='Intensidade ($10^2$ cps)'):
    """ Funcao que plota o terceiro grafico, onde é mostrado os valores de Counts retirado o background linear junto com a 
    curva de background shirley.
    """
    
    plt.plot(x,y1/100,sta,markersize=6,linewidth=3,markeredgecolor='black', alpha=0.5)
    plt.plot(x,y2/100,stb)
    plt.xlim(xli)

    plt.xlabel(xlab, fontsize=24)
    plt.ylabel (ylab, fontsize=24)
    plt.grid(True,color ='lightgray',linestyle='--',linewidth='1')
    plt.tick_params('both', labelsize=15)
    plt.savefig('trgraf_320eV.jpeg')
    
    return plt.show()
#####

def Lgraf(x, y, st='-k', yli=-0.1, xli=(400,610), xlab='BE (eV)', ylab='Intensidade ($10^2$ cps)', titl='XPS data'):
    """ Função que plota o ultimo grafico apos retirar todos os backgrounds    
    """    
    plt.figure(figsize=(10,10))
    plt.plot(x, y/100,st, markersize=6,linewidth=3, markerfacecolor='lightgray', markeredgecolor='black', alpha=0.5)

    plt.ylim(yli)
    plt.xlim(xli)

    plt.xlabel(xlab, fontsize=20)
    plt.ylabel (ylab, fontsize=20)
    plt.title(titl, fontsize=18 )
    plt.grid(True,color ='lightgray',linestyle='--',linewidth='1')
    plt.tick_params('both', labelsize=15)
    plt.savefig('lgraf_320eV.jpeg')
    
    return plt.show()
#####

def graf_phi(cy,dici,y='Ksi',st='-k'):
    """ Funcao que plota um grafico de uma coluna de um dataframe presente em um dicionario em função da coluna theta
    do mesmo dataframe, pra um vaolr fixo de phi (azimuths).
    ###########
    Parametros
    ----------
    cy : int-type
        O numero do cycle (que se relaciona diretamente com o valor fixo de phi que se deseja plotar) presente no
        dataframe que ira utilizar.
    dici: dic-type
        Um dicionario que contenha o dataframe de onde serao retirado os dados.
    y: str-type, padrao 'Ksi'
        O nome da coluna do dataframe que será utilizada no eixo das ordenadas.
    st: str-type, padrao '-k'
        O estilo do grafico a ser plotado.
    
    ###########
    """
    
    ex_phi=cy
    z=list()
    zz=list()
    for key in dici:
        #print(key)
        data_index=dici[key][0]
        data_index['Theta']=np.array(abs(data_index['Theta']))
        data_index=data_index.reindex(columns=['Cycle','Phi','Curve', 'Theta', 'Area','Psi','Ksi', 'Log_area'])
        data_index= data_index.sort_values(by=['Cycle', 'Phi'])
        q=np.arange(len(data_index))
        data_index[' ']=q
        data_index=data_index.set_index(' ')

        j=len(dici[key][0])
        for row in range(j):
            if data_index['Cycle'][row]==ex_phi:
                z.append(data_index['Theta'][row])
                zz.append(data_index[y][row])

    z=np.array(z)
    zz=np.array(zz)
    data_graf=pd.DataFrame(z, columns=['Theta'])
    data_graf[y]=zz
    data_graf= data_graf.sort_values(by=['Theta'])
    q=np.arange(len(data_graf))
    data_graf[' ']=q
    data_graf=data_graf.set_index(' ')
    #print(data_graf)
    z=data_graf['Theta']
    zz=data_graf[y]
    
    l=len(z)
    plt.plot(z,zz,st)
    plt.xlim(z[0]-1, z[l-1]+1)
    plt.xlabel('Theta (deg)', fontsize=15)
    plt.ylabel(y, fontsize=16)
    
    return plt.show()
#####

def graf_theta(tli,cur,dici,b,y='Ksi',st='-k'):
    """ Funcao que plota um grafico de uma coluna de um dataframe presente em um dicionario em função da coluna phi
    do mesmo dataframe, pra um vaolr fixo de theta (polar).
    ###########
    Parametros
    ----------
    tli : int-type
        O numero total de linhas presente no dataframe que ira utilizar (pode-se chegar a esse valor multiplicando a
        quantidade de cycles pela quantidade de curves). Esse valor sera utilizado como intervalo para execucao do 
        loop for. Um intervalo, desde que valido, tambem e admtido:
            { (inicio(opcional), fim, passo(opcional)), ex.: 1-(910); 2-(200,800); 3-(200,800,2) }
    cur : int-type
        O valor da curva (que se relaciona diretamente com o valor escolhido para fixar theta) presente no dataframe
        que ira utilizar. Nao exceder a quantidade de curvas presentes por cycle. Ex.: cada cycle do dataframe possui
            10 curves (os valore validos seriam de 0 a 9), o numero de curva 10, ou superior, nao sera aceito.
    dici: dic-type
        Um dicionario que contenha o dataframe de onde serao retirados os dados.
    b : str-type
        Keys do dicionario. Por padrao as keys sao os nomes dos arquivos de texo lidos pelo programa. Pode-se acessar
        as keys de um dicionario com o comando <nome do dicionario>.keys(). Ex.: Arquivos2.keys()
    y : str-type, padrao 'Ksi'
        O nome da coluna do dataframe que será utilizada no eixo das ordenadas.
    st : str-type, padrao '-k'
        O estilo do grafico a ser plotado.
    
    ###########
    """
    
    ex_theta=cur
    z=list()
    zz=list()
    for row in range(tli):
        if dici[b][0]['Curve'][row]==ex_theta:
            z.append(dici[b][0]['Phi'][row])
            zz.append(dici[b][0][y][row])
    z=np.array(z)
    zz=np.array(zz)
    #print(z)
    
    plt.plot(z,zz,st)
    plt.xlim((z[0]-1, z[-1]+1))
    plt.xlabel('Phi (deg)', fontsize=15)
    plt.ylabel(y, fontsize=16)

    return plt.show()
#####


### Main()

In [ ]:
%%time
## comando para medir o tempo de execução do programa. Exclusivo para o Jupyter notebook

################################## funcao principal (main) ######################################

arq= glob.glob('XPD_Cu_2_snapshot_th*.xy')   ## armazena os nomes dos arquivos a serem lidos na array 'arq', nessa forma
                                             ## só funciona no jupyter, sendo que os arquivos a serem lidos devem estar
                                             ## na mesma pasta deste script.
        
## No formato abaixo, ela funciona de forma geral, pois é especificado o endereço da pasta que contem os arquivos.                                             
## arq= glob.glob('C:\Users\luiz.renato\Google Drive\Bolsa Verao 2018\Jupyter\Dados reais\FG664_XPD_T*.xy')
## print(arq)

## declara dicionarios que serao usados para armazenar os dados ao fim do programa
Arquivos1 = dict()


## For para ler cada arquivo de dados 
for a in arq:
    file_str= open(a,'r').read()
    getelmt = re.split('Region: *', file_str)[1:]
    ## separa os dados dos elementos diferentes presentes no arquivo
    elmt1= getelmt[0]
    print(a)
    
    ind=-1
    newp=list()
    newt=list()

    ## For para ler os dados dos elementos ja separados
    for i in [elmt1]:

        ##coletando informacao do valor dos angulos
        azimuths = re.findall('Parameter: \"azimuth \[deg\]\" = (-?[0-9]*)', i)
#        #polars = re.findall('Parameter: \"polar \[deg\]\" = (-?[0-9]*)', i)
        theta = int(a[20:22])*(-1)
                
        phi = np.array(azimuths).astype('int')
#        #theta = np.array(polars).astype('int')
        
        ## Separando as informacoes dos dados em si
        medidas = re.split('Cycle: [0-9]*, Curve: [0-9]*', i)[1:]
        cycle_curve = re.findall('(Cycle: [0-9]*, Curve: [0-9]*)', i)
        NonEO= re.findall('NonEnergyOrdinate: (-?[0-9.]*)',i)
        
        ## Declarando listas para armazenamento provisorio de dados
        aaa=list()
        aab=list()
        aac=list()
        aaf=list()
        k=0
        for cc,non,med in zip(cycle_curve,NonEO,medidas):
            cycle = int(re.findall('Cycle: ([0-9]*)',cc)[0])
            curve = int(re.findall('Curve: ([0-9]*)',cc)[0])
            if curve==0:
                ind+=1
            ## Para aleitura desta coleção de arquivos especificamente este if permite quebrar em determinado ciclo a coleta
            ## de dados do arquivo com o nome especificado abaixo
            if a=='XPD_Cu_2_snapshot_th51.xy':
                if cycle==61: 
                    break
            if a == 'XPD_Cu_2_snapshot_th70.xy':
                if cycle<=9:
#                    print('ok')
                    continue
                elif curve<=1:
                    continue
                    
            NEO = float(non) ## transforma para float os dados presentes em NonEO
                      
            nums = re.findall('(-?[0-9.]*  -?[0-9.]*)',med) ## Busca pelos dados na variavel 'med'
#            print(nums)
#            print(cycle, curve)
            be = list()
            counts = list()
            
            ## For para separar os dados em suas listas proprias
            for n in nums:
                try:
                    be.append(float(n.split('  ')[0]))
                    counts.append(float(n.split('  ')[1]))
                except:
                    break
            ## armazenas os dados abaixo em uma unica lista
            aaa.append([cycle, curve, NEO, be, counts])
            newp.append(phi[ind])
            newt.append(theta)
#            print(be)
#            print(counts)
                
            ## passa os dados de lista para uma array do numpy    
            BE= np.array(be)
            Counts= np.array(counts)
#            print(type(BE), type(Counts))
#            print(len(be),len(counts) )
                
            ## grafico inicial, sem tatamento algum dos dados
#            Stgraf(BE, Counts,st='ok', xli=(BE[0]-0.5, BE[-1]+0.5)) 
#            input("Press Enter to continue...")

            ## Chama a funcao que retira o background linear
#            Bline = linearBG(BE,Counts)
#            print(len(Bline))

            ## grafico que plota a linha de backgroud linear junto com os dados iniciais
#            Segraf(BE, Bline, Counts, xli=(BE[0]-0.5, BE[-1]+0.5))
#            input("Press Enter to continue...")

            ## subtracao do BG linear do counts
#            LCounts = Counts-Bline

            ## tira o backgroud shirley, neste caso direto dos dados e subtrai do counts
            Bsh = shirley(BE,Counts, tol=1e-5, maxit=40)
            SCounts = Counts - Bsh
#            print(Bsh)

            ## plota o grafico com a linha de BG shirley junto com os dados sem tratamento
#            Tgraf=(BE, Counts, Bsh)
#            input("Press Enter to continue...")

            ## Plota o grafico final, ja retirado o BG dos dados.
#            Lgraf(BE, SCounts, xli=(BE[0]-0.5, BE[-1]+0.5))
#            input("Press Enter to continue...")
            
            #plota gráficos de cycles e curves específicos para um determinado arquivo
#            if a=='XPD_Cu_2_snapshot_th13.xy':
#                if curve==4:
#                    if cycle==31:
#                        Stgraf(BE, Counts,st='ok', xli=(BE[0]-0.5, BE[-1]+0.5))
#                        Tgraf(BE, Counts, Bsh, sta='-k', stb='-r', xli=(BE[0]+2, BE[-1]-2),xlab='BE (eV)', ylab='Intensidade ($10^2$ cps)')
#                        Lgraf(BE, SCounts,yli=-0.1, xli=(BE[0]-0.5, BE[-1]+0.5))


            ## integra os dados para calcular a area sob a cuva
            area = integrate.simps(SCounts,dx=0.05)
#            aab.append(Bline)
            aac.append(Bsh)
            aaf.append(area)
            #print(area)              
               
        ## transforma as listas em arrays do numpy para gerar dataframes com os dados     
        aa= np.array(aaa, dtype=object)
        newp=np.array(newp)
        newt=np.array(newt)
        
               
        ######### Data Frame com  informações separadas em colunas e separadas por arquivo de dados ###########
        if i==elmt1:
            Inf1 = pd.DataFrame(aa, columns=['Cycle', 'Curve', 'NonEnerOrd', 'BE', 'Counts'])
            Inf1['Phi'] = newp
            Inf1['Theta'] = newt
            Inf1= Inf1.reindex(columns=['Cycle', 'Curve', 'NonEnerOrd', 'Phi', 'Theta', 'BE', 'Counts'])
#            Inf1['BGL'] = aab
            Inf1['BSh'] = aac
            Inf1['Area'] = aaf

        
    ### Dicionario com informacoes de todos os arquivos
    ## A chave para selecionar uma colecao de dados e o nome completo do arquivo lido
    ## na duvida utilize o comando Arquivos1.keys()
    Arquivos1[a]=[Inf1]


In [ ]:
%%time

### For que organiza os dados em um novo dataframe para a plotagem de graficos mais refinados
## Dicionario para armazenar os dados ao final do programa
Arquivos2 = dict()

## Tratamento especifico para esse arquivo de dados
for key in Arquivos1:
    if key=='XPD_Cu_2_snapshot_th51_2.xy':
        Arquivos1[key][0]['Cycle']=np.array(Arquivos1[key][0]['Cycle']+60)
    print(key)
    Data0=Arquivos1[key][0]
    Data1=Data0.loc[:, ['Curve','Cycle','Phi','Area']]
    Data1['Theta']=Data0['Theta'] + Data0['NonEnerOrd']
    Data1=Data1.reindex(columns=['Curve', 'Theta','Cycle','Phi', 'Area'])
    Data2= Data1.sort_values(by=['Curve', 'Theta'])
    q=np.arange(len(Data2))
    Data2[' ']=q
    Data2=Data2.set_index(' ')
        
    ## tira a media dos valores da area para realizar a normalizacao dos dados
    soma=0
    N=0
    nor=list()
    l_val2=list()
    angle2=list()
    interval=range(len(Data2))
    for lin in interval:
        if lin == 0:
            angle2.append(Data2['Theta'][lin])
            soma=soma+Data2['Area'][lin]
            nor.append(Data2['Area'][lin])
            N+=1
        elif lin==interval[-1]:
            soma=soma+Data2['Area'][lin]
            nor.append(Data2['Area'][lin])
            N+=1
            pico=np.amax(nor)
            vale=np.amin(nor)
            l_val2.append([soma,N,pico,vale])
        elif Data2['Theta'][lin] == Data2['Theta'][lin-1]:
            soma=soma+Data2['Area'][lin]
            nor.append(Data2['Area'][lin])
            N+=1
        elif Data2['Theta'][lin] == Data2['Theta'][lin+1]:
            angle2.append(Data2['Theta'][lin])
            pico=np.amax(nor)
            vale=np.amin(nor)
            l_val2.append([soma,N,pico,vale])
            N=0
            soma=0
            nor=list()
            soma=soma+Data2['Area'][lin]
            nor.append(Data2['Area'][lin])
            N+=1
#        print(soma)

    interval=range(len(l_val2))
    media2=list()
    for v in interval:
        M = l_val2[v][0]/l_val2[v][1]
#        print(M)
        media2.append(M)
    Media2=np.array([media2, angle2])
        
    ## realisa a normalizacao dos dados pela média dos valores para um mesmo theta e armazena no dataframe
    val=0
    norma=list()
    interval=range(len(Data2))
    for lin in interval:
        if Data2['Theta'][lin]==Media2[1][val]:
            psi= (Data2['Area'][lin] - Media2[0][val])/Media2[0][val]
            norma.append(psi)
            
            if Data2['Theta'][lin] != Data2['Theta'][interval[-1]] and Data2['Theta'][lin] != Data2['Theta'][lin+1]:
                val+=1
        else:
            print('Erro ao normalisar')
            print(lin, 'err', val)
    Psi=np.array(norma)
    Data2['Psi']=Psi
    
    ## realisa a normalizacao dos dados com base nos valores maximos e minimos para um mesmo theta e armazena no dataframe
    val=0
    norma=list()
    interval=range(len(Data2))
    for lin in interval:
        if Data2['Theta'][lin]==Media2[1][val]:
            ksi= (Data2['Area'][lin] - l_val2[val][3])/(l_val2[val][2]-l_val2[val][3])
            norma.append(ksi)
            
            if Data2['Theta'][lin] != Data2['Theta'][interval[-1]] and Data2['Theta'][lin] != Data2['Theta'][lin+1]:
                    val+=1
        else:
            print('Erro ao normalisar')
            print(lin, 'err', val)
    Ksi=np.array(norma)
    Data2['Ksi']=Ksi
    
    data_op= np.array(Data2['Area'])
    data_op= np.log10(data_op)
    Data2['Log_area']=data_op    
     
            
    Arquivos2[key]=[Data2, Media2]
        

In [ ]:
## Utilizacao das funcoes que plotam graficos em relacao a phi ou a theta
#graf_phi(30,Arquivos2,y='Area',st='-ok')
#graf_theta(910,4,Arquivos2,'XPD_Cu_2_snapshot_th13.xy',y='Ksi',st='-k')

In [ ]:
## comando para imprimir todo o dataframe do arquivo especificado

#b='XPD_Cu_2_snapshot_th32.xy'
#j=len(Arquivos2[b][0])
#print('Curve','\t','Theta','\t\t','Cycle','\t','Phi','\t\t','Area','\t\t','Psi','\t\t','Ksi')
#for row in range(j):
#    print(Arquivos2[b][0]['Curve'][row],'\t',Arquivos2[b][0]['Theta'][row],'\t',Arquivos2[b][0]['Cycle'][row],'\t',
#          Arquivos2[b][0]['Phi'][row],'\t',Arquivos2[b][0]['Area'][row],'\t',Arquivos2[b][0]['Psi'][row],'\t',
#          Arquivos2[b][0]['Ksi'][row])


In [ ]:
## tratamento dos dados presente nos dicionarios para a plotagem de difratogramas
## leitura do dicionario
Data01=Arquivos2['XPD_Cu_2_snapshot_th13.xy'][0]
Data02=Arquivos2['XPD_Cu_2_snapshot_th32.xy'][0]
Data03=Arquivos2['XPD_Cu_2_snapshot_th51.xy'][0]
Data04=Arquivos2['XPD_Cu_2_snapshot_th51_2.xy'][0]
Data05=Arquivos2['XPD_Cu_2_snapshot_th70.xy'][0]

## transforma os dados em uma array do numpy
phi_data01 = np.array(Data01['Phi'])
theta_data01 = np.array(Data01['Theta'])
#area_data01 = np.array(Data01['Area'])
#area_data01 = np.array(Data01['Psi'])
area_data01 = np.array(Data01['Ksi'])
#area_data01 = np.array(Data01['Log_area'])

phi_data02 = np.array(Data02['Phi'])
theta_data02 = np.array(Data02['Theta'])
#area_data02 = np.array(Data02['Area'])
#area_data02 = np.array(Data02['Psi'])
area_data02 = np.array(Data02['Ksi'])
#area_data02 = np.array(Data02['Log_area'])

phi_data03 = np.array(Data03['Phi'])
theta_data03 = np.array(Data03['Theta'])
#area_data03 = np.array(Data03['Area'])
#area_data03 = np.array(Data03['Psi'])
area_data03 = np.array(Data03['Ksi'])
#area_data03 = np.array(Data03['Log_area'])

phi_data04 = np.array(Data04['Phi'])
theta_data04 = np.array(Data04['Theta'])
#area_data04 = np.array(Data04['Area'])
#area_data04 = np.array(Data04['Psi'])
area_data04 = np.array(Data04['Ksi'])
#area_data04 = np.array(Data04['Log_area'])

phi_data05 = np.array(Data05['Phi'])
theta_data05 = np.array(Data05['Theta'])
#area_data05 = np.array(Data05['Area'])
#area_data05 = np.array(Data05['Psi'])
area_data05 = np.array(Data05['Ksi'])
#area_data05 = np.array(Data05['Log_area'])

## reorganiza o modo como os dados sao apresentados utilizando o reshape
phi_data01=np.reshape(phi_data01, (10,91))
theta_data01=np.reshape(theta_data01, (10,91))
area_data01=np.reshape(area_data01, (10,91))

phi_data02=np.reshape(phi_data02, (10,91))
theta_data02=np.reshape(theta_data02, (10,91))
area_data02=np.reshape(area_data02, (10,91))

phi_data03=np.reshape(phi_data03, (10,61))
theta_data03=np.reshape(theta_data03, (10,61))
area_data03=np.reshape(area_data03, (10,61))

phi_data04=np.reshape(phi_data04, (10,31))
theta_data04=np.reshape(theta_data04, (10,31))
area_data04=np.reshape(area_data04, (10,31))

phi_data05=np.reshape(phi_data05, (8,81))
theta_data05=np.reshape(theta_data05, (8,81))
area_data05=np.reshape(area_data05, (8,81))

## Plotagem do grafico em forma de "pizza"
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111, projection='polar')

ma=1.1  ## vallor maximo para a escala de cor
mi=0.2  ## valor minimo para a escala de cor
xpd01= ax.contourf(np.radians(phi_data01), np.abs(theta_data01),area_data01, 100, vmin=mi, vmax=ma, cmap=cm.gray)
xpd02= ax.contourf(np.radians(phi_data02), np.abs(theta_data02),area_data02, 100, vmin=mi, vmax=ma, cmap=cm.gray)
xpd03 = ax.contourf(np.radians(phi_data03), np.abs(theta_data03),area_data03, 100, vmin=mi, vmax=ma, cmap=cm.gray)
xpd04 = ax.contourf(np.radians(phi_data04), np.abs(theta_data04),area_data04, 100, vmin=mi, vmax=ma, cmap=cm.gray)
xpd05 = ax.contourf(np.radians(phi_data05), np.abs(theta_data05),area_data05, 100, vmin=mi, vmax=ma, cmap=cm.gray)

ax.set_yticklabels([]) ## remove os valores da coordenada radial
#ax.set_xticklabels([])
ax.set_rmax(84) ## define valor maximo da coordenada radial
ax.set_rmin(0) ## define valor minimo da coordenada radia
ax.grid(True); ## remove o grid
#ax.set_title('a) Experimento', fontsize=24) ## Insere o título do grafico
#fig.colorbar(xpd01)  ##
#fig.colorbar(xpd02)  ##
#fig.colorbar(xpd03)  ## Mostra a barra da escala de cores do grafico
#fig.colorbar(xpd04)  ##
#fig.colorbar(xpd05)  ##

#plt.savefig('XPD_Cu_2_snapshot_theta_ksinorm.png')  ## Comando para salvar a figura
plt.show()
#input("Press enter to continue...")
    

In [ ]:
#graf_phi(49,Arquivos2,y='Ksi',st='--k')
#graf_phi(10,Arquivos2,y='Ksi',st='--k')

In [ ]:
## Sequencia para gerar o difratograma unindo todos os dados em um unico arquivo

Data0=Arquivos2['XPD_Cu_2_snapshot_th13.xy'][0]
Data00=Arquivos2['XPD_Cu_2_snapshot_th32.xy'][0]
Data000=Arquivos2['XPD_Cu_2_snapshot_th51.xy'][0]
Data0000=Arquivos2['XPD_Cu_2_snapshot_th51_2.xy'][0]
Data00000=Arquivos2['XPD_Cu_2_snapshot_th70.xy'][0]


phi_data0 = np.array(Data0['Phi']+60)
theta_data0 = np.array(abs(Data0['Theta']))
#area_data0 = np.array(Data0['Area'])
#area_data0 = np.array(Data0['Psi'])
area_data0 = np.array(Data0['Ksi'])
#area_data0 = np.array(Data0['Log_area'])
curve_data0 = np.array(Data0['Curve'])

phi_data00 = np.array(Data00['Phi']+60)
theta_data00 = np.array(abs(Data00['Theta']))
#area_data00 = np.array(Data00['Area'])
#area_data00 = np.array(Data00['Psi'])
area_data00 = np.array(Data00['Ksi'])
#area_data00 = np.array(Data00['Log_area'])
curve_data00 = np.array(Data00['Curve']+10)

phi_data000 = np.array(Data000['Phi']+60)
theta_data000 = np.array(abs(Data000['Theta']))
#area_data000 = np.array(Data000['Area'])
#area_data000 = np.array(Data000['Psi'])
area_data000 = np.array(Data000['Ksi'])
#area_data000 = np.array(Data000['Log_area'])
curve_data000 = np.array(Data000['Curve']+20)

phi_data0000 = np.array(Data0000['Phi']+60)
theta_data0000 = np.array(abs(Data0000['Theta']))
#area_data0000 = np.array(Data0000['Area'])
#area_data0000 = np.array(Data0000['Psi'])
area_data0000 = np.array(Data0000['Ksi'])
#area_data0000 = np.array(Data0000['Log_area'])
curve_data0000 = np.array(Data0000['Curve']+20)

phi_data00000 = np.array(Data00000['Phi']+60)
theta_data00000 = np.array(abs(Data00000['Theta']))
#area_data00000 = np.array(Data00000['Area'])
#area_data00000 = np.array(Data00000['Psi'])
area_data00000 = np.array(Data00000['Ksi'])
#area_data00000 = np.array(Data00000['Log_area'])
curve_data00000 = np.array(Data00000['Curve']+30)

#################################################
## Para essa sequncia de arquivos especificamente, ha a necessidade de unir dpois desses arquivos, 
## pois um completa o outro. A sequencia esta abaixo.
phi_data000=np.reshape(phi_data000, (10,60))
theta_data000=np.reshape(theta_data000, (10,60))
area_data000=np.reshape(area_data000, (10,60))
curve_data000 = np.reshape(curve_data000, (10,60))
phi_data0000=np.reshape(phi_data0000, (10,31))
theta_data0000=np.reshape(theta_data0000, (10,31))
area_data0000=np.reshape(area_data0000, (10,31))
curve_data0000 = np.reshape(curve_data0000, (10,31))

phi= np.concatenate((phi_data000, phi_data0000), axis=1)
theta= np.concatenate((theta_data000, theta_data0000), axis=1)
area= np.concatenate((area_data000, area_data0000), axis=1)
curve= np.concatenate((curve_data000, curve_data0000), axis=1)

phi0= np.reshape(phi, (910))
theta0= np.reshape(theta, (910))
area0= np.reshape(area, (910))
curve0= np.reshape(curve, (910))

####################################################
## Concatenando os dados em um unico array, separados, e claro, por tipo de dado
phi_data = np.concatenate((phi_data0,phi_data00,phi0), axis=0)
theta_data = np.concatenate((theta_data0,theta_data00,theta0), axis=0)
area_data = np.concatenate((area_data0, area_data00, area0), axis=0)
curve_data = np.concatenate((curve_data0, curve_data00, curve0), axis=0)

Data_org = pd.DataFrame(curve_data, columns=['Curve'])
Data_org['Theta']=theta_data
Data_org['Phi']=phi_data
Data_org['Area']=area_data
Data_org= Data_org.sort_values(by=['Theta', 'Phi'], ascending=True)
q=np.arange(len(Data_org))
Data_org[' ']=q
Data_org=Data_org.set_index(' ')

phi_data = np.array(Data_org['Phi'])
theta_data = np.array(Data_org['Theta'])
area_data = np.array(Data_org['Area'])

##################
## Reshape para a formacao do difratograma
phi_data=np.reshape(phi_data, (30,91))
theta_data=np.reshape(theta_data, (30,91))
area_data=np.reshape(area_data, (30,91))

## Sequencia para a plotagem do difratograma  
fig = plt.figure(figsize=(15,15))
ax = plt.subplot(111, projection='polar')
xpd=ax.contourf(np.radians(phi_data), np.abs(theta_data),area_data, 100, vmin=0, vmax=1, cmap=cm.gray)

ax.set_yticklabels([])# remove os valores da coordenada radial
#ax.set_xticklabels([])
ax.set_rmax(90) # define valor maximo da coordenada radial
ax.set_rmin(0) #define valor minimo da coordenada radia
ax.grid(True); #remove o grid
fig.colorbar(xpd)

#plt.savefig('XPD_Cu_2_snapshot_phi.jpg')  ## Salvar o difratograma em figura
plt.show()
#input("Press enter to continue...")
